In [1]:
import requests
import os 
import pandas as pd
import time 
import json
from pandas.io.json import json_normalize  


#get auth token 
KEY = os.environ['PETFINDER_KEY']
SECRET = os.environ['PETFINDER_SECRET']

def GetBearerToken(KEY,SECRET):
    data = {
    'grant_type': 'client_credentials',
    'client_id': KEY,
    'client_secret': SECRET
      }
    response = requests.post('https://api.petfinder.com/v2/oauth2/token', data=data)
    if response.reason != 'OK':
        print('failed to get token. check credentials')
        print('response code:', response.status_code)
        print('response reason:', response.reason)
    else:
        print('new token received')
        TOKEN = json.loads(response.text)['access_token']
        new_header = {
                'Authorization': 'Bearer {}'.format(TOKEN),}
    return new_header

    
header = GetBearerToken(KEY,SECRET)

type = 'Dog'
location = 85017    #kansas city 64106, #milwaukee 53202      #78401 corpuscristi                   #omaha #oklahoma city   
limit = 100
distance = 100 
status = 'adopted'
outfile = 'Phoenix_adopted_20k_Jan_29.csv'

respdf = pd.DataFrame()

for i in range(200):
    page = i + 1 
    url = f'https://api.petfinder.com/v2/animals?type={type}&location={location}&limit={limit}&page={page}&distance={distance}&status={status}'
    print(i)
    response = requests.get(url, headers=header)
    my_dict =json.loads(response.text)
    animals = my_dict.get('animals')
    df = pd.io.json.json_normalize(animals)
    respdf = respdf.append(df)
    time.sleep(1)

respdf.to_csv(path_or_buf = outfile)



new token received
0


TypeError: 'NoneType' object is not iterable

In [4]:
respdf.to_csv(path_or_buf = outfile)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from petdata_functions import CleanPetData
from petdata_functions import ReformatDates


#read in data 
chidat = pd.read_csv("Chicago_adopted_20k_Jan_21.csv") 
chidat['City'] = 'Chicago'

houstdat = pd.read_csv("Houston_adopted_20k_Jan_21.csv") 
houstdat['City'] = 'Houston'

elpdat = pd.read_csv("ElPaso_adopted_20k_Jan_21.csv") 
elpdat['City'] = 'ElPaso'

Minndat = pd.read_csv("Chicago_adopted_20k_Jan_21.csv") 
Minndat['City'] = 'Minne'

dendat = pd.read_csv("Denver_adopted_20k_Jan_21.csv") 
dendat['City'] = 'Denver'

stldat = pd.read_csv("StL_adopted_20k_Jan_21.csv")
stldat['City'] = 'StLouis'

inddat = pd.read_csv("Indy_adopted_20k_Jan_21.csv")
inddat['City'] = 'Indy'

petdat = pd.concat([chidat,houstdat,elpdat,Minndat,dendat, stldat, inddat])


petdat = CleanPetData(petdat)

#petdat = ReformatDates(petdat)
    
fmt ='%Y-%m-%dT%H:%M:%S+0000'
petdat['published_at'] =  pd.to_datetime(petdat['published_at'], format=fmt)
petdat['status_changed_at'] =  pd.to_datetime(petdat['status_changed_at'], format=fmt)
petdat['time_diff'] = petdat['status_changed_at'] - petdat['published_at']
petdat['target_hour'] = petdat.time_diff.astype('timedelta64[h]')
    